In [ ]:
# spenders_df = pd.DataFrame({"SN": [],
#                            "Purchase Count": [],
#                            "Average Purchase Price": [],
#                           "Total Purchase Value": []})


# popular_df = pd.DataFrame({"Item ID": [],
#                            "Item Name": [],
#                            "Purchase Count": [],
#                            "Item Price": [],
#                           "Total Purchase Value": []})

# profitable_df = pd.DataFrame({"Item ID": [],
#                            "Item Name": [],
#                            "Purchase Count": [],
#                            "Item Price": [],
#                           "Total Purchase Value": []})